# Subject Specific Classification 
Because the previous results failed to perform with regards to the null model during the last submission deadline, let's try to use subject specific data

In [52]:
import tensorflow as tf
# tf.enable_eager_execution()
# Helperfunctions to make your feature definition more readable

In [53]:
import tf_mapper
from importlib import reload
reload(tf_mapper)
from tf_mapper import get_batched_dataset

In [54]:
train_indices, valid_indices, test_indices = tf_mapper.get_indices()

In [55]:
all_subjects = tf_mapper.get_subjects()

In [56]:
def get_model():
    num_cnn_layers = 5
    num_lstm_layers = 1
    num_lin_layers = 2
    inputLayer = tf.keras.layers.Input((1500, 3))
    x = inputLayer


    for i in range(num_cnn_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(16, (3,), padding="same")(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.MaxPool1D((2,))(x)
    for j in range(num_lstm_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.CuDNNLSTM(256, return_sequences=True)(x)
        x = tf.keras.layers.LeakyReLU()(x)


    x = tf.keras.layers.Flatten(name="flatten_encoder_lstm")(x)
    x = tf.keras.layers.Dense(200)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x_shared_flattened = x

    #one_off
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(256)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_on_off = tf.keras.layers.ReLU(name="on_off", max_value=4)(x)

    #tremor
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(100)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_dyskinesia = tf.keras.layers.ReLU(name="dyskinesia", max_value=4)(x)

    #montage classify
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(100)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_tremor = tf.keras.layers.ReLU(name="tremor", max_value=4)(x)



    model = tf.keras.Model(inputs=inputLayer, outputs=[x_on_off, x_dyskinesia, x_tremor])
    model.compile("adam", loss=["mean_squared_error", "mean_squared_error","mean_squared_error", ])
    return model

In [57]:
all_subjects = tf_mapper.get_subjects()
all_subjects

[1004,
 1006,
 1007,
 1019,
 1020,
 1023,
 1032,
 1034,
 1038,
 1039,
 1043,
 1044,
 1046,
 1048,
 1049,
 1051]

In [58]:
import pickle as pkl

In [60]:
reload(tf_mapper)
for subject in all_subjects:
    model = get_model()
    train_indices, valid_indices, test_indices = tf_mapper.get_splits_per_subject(subject)
    train_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(train_indices), batch_size=128, map_example=tf_mapper.map_example_to_simple_train, max_queue_size=20, cache=True)
    valid_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(valid_indices), batch_size=512, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(test_indices), batch_size=512, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(f"/n/scratch2/ms994/cnnlstm3{subject}.h5", save_best_only=True, verbose=True)
    reduceLR = tf.keras.callbacks.ReduceLROnPlateau(patience=4, factor=0.2, verbose=True)
    earlyStopping = tf.keras.callbacks.EarlyStopping(patience=20, min_delta=0.01)
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda i, lr: lr*0.8)
    history = model.fit(train_data, steps_per_epoch=250, epochs=200, validation_data=valid_data, validation_steps=100, callbacks=[modelCheckpoint, reduceLR, earlyStopping])
    pkl.dump(history.history, open(f"/n/scratch2/ms994/cnnlstm3{subject}_history.pkl", "wb"))

Epoch 1/200
248/250 [============================>.] - ETA: 0s - loss: 2.5317 - on_off_loss: 1.0682 - dyskinesia_loss: 0.8814 - tremor_loss: 0.5821
Epoch 00001: val_loss improved from inf to 8.18358, saving model to /n/scratch2/ms994/cnnlstm31004.h5
250/250 [==============================] - 94s 376ms/step - loss: 2.5265 - on_off_loss: 1.0666 - dyskinesia_loss: 0.8791 - tremor_loss: 0.5808 - val_loss: 8.1836 - val_on_off_loss: 3.5910 - val_dyskinesia_loss: 2.3590 - val_tremor_loss: 2.2336
Epoch 2/200
249/250 [============================>.] - ETA: 0s - loss: 1.6119 - on_off_loss: 0.7739 - dyskinesia_loss: 0.4568 - tremor_loss: 0.3813
Epoch 00002: val_loss improved from 8.18358 to 6.87030, saving model to /n/scratch2/ms994/cnnlstm31004.h5
250/250 [==============================] - 11s 45ms/step - loss: 1.6109 - on_off_loss: 0.7734 - dyskinesia_loss: 0.4562 - tremor_loss: 0.3813 - val_loss: 6.8703 - val_on_off_loss: 3.4459 - val_dyskinesia_loss: 1.6435 - val_tremor_loss: 1.7809
Epoch 3/2

Epoch 19/200
248/250 [============================>.] - ETA: 0s - loss: 0.1026 - on_off_loss: 0.0352 - dyskinesia_loss: 0.0291 - tremor_loss: 0.0383
Epoch 00019: val_loss did not improve from 6.87030
250/250 [==============================] - 11s 43ms/step - loss: 0.1026 - on_off_loss: 0.0352 - dyskinesia_loss: 0.0290 - tremor_loss: 0.0384 - val_loss: 8.4675 - val_on_off_loss: 3.4230 - val_dyskinesia_loss: 2.4088 - val_tremor_loss: 2.6357
Epoch 20/200
249/250 [============================>.] - ETA: 0s - loss: 0.1014 - on_off_loss: 0.0339 - dyskinesia_loss: 0.0290 - tremor_loss: 0.0386
Epoch 00020: val_loss did not improve from 6.87030
250/250 [==============================] - 11s 43ms/step - loss: 0.1015 - on_off_loss: 0.0338 - dyskinesia_loss: 0.0291 - tremor_loss: 0.0385 - val_loss: 8.4551 - val_on_off_loss: 3.4254 - val_dyskinesia_loss: 2.3955 - val_tremor_loss: 2.6343
Epoch 21/200
249/250 [============================>.] - ETA: 0s - loss: 0.1040 - on_off_loss: 0.0350 - dyskinesia_

Epoch 15/200
248/250 [============================>.] - ETA: 0s - loss: 0.0641 - on_off_loss: 0.0122 - dyskinesia_loss: 0.0371 - tremor_loss: 0.0147
Epoch 00015: val_loss did not improve from 6.09826
250/250 [==============================] - 11s 43ms/step - loss: 0.0641 - on_off_loss: 0.0122 - dyskinesia_loss: 0.0371 - tremor_loss: 0.0147 - val_loss: 6.3070 - val_on_off_loss: 1.8748 - val_dyskinesia_loss: 2.2224 - val_tremor_loss: 2.2098
Epoch 16/200
249/250 [============================>.] - ETA: 0s - loss: 0.0645 - on_off_loss: 0.0119 - dyskinesia_loss: 0.0380 - tremor_loss: 0.0147
Epoch 00016: val_loss did not improve from 6.09826

Epoch 00016: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
250/250 [==============================] - 11s 43ms/step - loss: 0.0646 - on_off_loss: 0.0119 - dyskinesia_loss: 0.0380 - tremor_loss: 0.0147 - val_loss: 6.3895 - val_on_off_loss: 1.8888 - val_dyskinesia_loss: 2.2708 - val_tremor_loss: 2.2298
Epoch 17/200
249/250 [===========

Epoch 9/200
249/250 [============================>.] - ETA: 0s - loss: 1.3244 - on_off_loss: 0.5860 - dyskinesia_loss: 0.3839 - tremor_loss: 0.3544
Epoch 00009: val_loss did not improve from 2.71274

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
250/250 [==============================] - 18s 71ms/step - loss: 1.3246 - on_off_loss: 0.5861 - dyskinesia_loss: 0.3838 - tremor_loss: 0.3547 - val_loss: 3.3565 - val_on_off_loss: 1.2816 - val_dyskinesia_loss: 1.0826 - val_tremor_loss: 0.9922
Epoch 10/200
249/250 [============================>.] - ETA: 0s - loss: 0.9526 - on_off_loss: 0.3787 - dyskinesia_loss: 0.2934 - tremor_loss: 0.2806
Epoch 00010: val_loss did not improve from 2.71274
250/250 [==============================] - 11s 44ms/step - loss: 0.9519 - on_off_loss: 0.3785 - dyskinesia_loss: 0.2931 - tremor_loss: 0.2803 - val_loss: 3.2909 - val_on_off_loss: 1.2515 - val_dyskinesia_loss: 1.0357 - val_tremor_loss: 1.0036
Epoch 11/200
248/250 [===========

Epoch 2/200
249/250 [============================>.] - ETA: 0s - loss: 1.2858 - on_off_loss: 0.4193 - dyskinesia_loss: 0.3024 - tremor_loss: 0.5641
Epoch 00002: val_loss did not improve from 4.87633
250/250 [==============================] - 11s 45ms/step - loss: 1.2850 - on_off_loss: 0.4188 - dyskinesia_loss: 0.3025 - tremor_loss: 0.5636 - val_loss: 6.4221 - val_on_off_loss: 2.8788 - val_dyskinesia_loss: 1.7793 - val_tremor_loss: 1.7640
Epoch 3/200
249/250 [============================>.] - ETA: 0s - loss: 0.8186 - on_off_loss: 0.2775 - dyskinesia_loss: 0.1838 - tremor_loss: 0.3573
Epoch 00003: val_loss did not improve from 4.87633
250/250 [==============================] - 11s 45ms/step - loss: 0.8181 - on_off_loss: 0.2775 - dyskinesia_loss: 0.1835 - tremor_loss: 0.3571 - val_loss: 6.3096 - val_on_off_loss: 3.3561 - val_dyskinesia_loss: 1.5047 - val_tremor_loss: 1.4488
Epoch 4/200
249/250 [============================>.] - ETA: 0s - loss: 0.5660 - on_off_loss: 0.2000 - dyskinesia_los

...

Epoch 7/200
248/250 [============================>.] - ETA: 0s - loss: 0.1020 - on_off_loss: 0.0382 - dyskinesia_loss: 0.0136 - tremor_loss: 0.0502
Epoch 00007: val_loss did not improve from 6.99700
250/250 [==============================] - 11s 46ms/step - loss: 0.1015 - on_off_loss: 0.0379 - dyskinesia_loss: 0.0135 - tremor_loss: 0.0500 - val_loss: 7.3610 - val_on_off_loss: 3.6170 - val_dyskinesia_loss: 1.4543 - val_tremor_loss: 2.2897
Epoch 8/200
249/250 [============================>.] - ETA: 0s - loss: 0.0815 - on_off_loss: 0.0300 - dyskinesia_loss: 0.0106 - tremor_loss: 0.0409
Epoch 00008: val_loss did not improve from 6.99700
250/250 [==============================] - 11s 46ms/step - loss: 0.0813 - on_off_loss: 0.0299 - dyskinesia_loss: 0.0106 - tremor_loss: 0.0409 - val_loss: 7.6377 - val_on_off_loss: 3.7011 - val_dyskinesia_loss: 1.5337 - val_tremor_loss: 2.4029
Epoch 9/200
248/250 [============================>.] - ETA: 0s - loss: 0.0616 - on_off_loss: 0.0217 - dyskinesia_los

Epoch 25/200
249/250 [============================>.] - ETA: 0s - loss: 0.0446 - on_off_loss: 0.0176 - dyskinesia_loss: 0.0062 - tremor_loss: 0.0208
Epoch 00025: val_loss did not improve from 6.99700

Epoch 00025: ReduceLROnPlateau reducing learning rate to 3.200000264769187e-07.
250/250 [==============================] - 12s 46ms/step - loss: 0.0445 - on_off_loss: 0.0175 - dyskinesia_loss: 0.0062 - tremor_loss: 0.0208 - val_loss: 7.4462 - val_on_off_loss: 3.6214 - val_dyskinesia_loss: 1.4633 - val_tremor_loss: 2.3615
Epoch 1/200
249/250 [============================>.] - ETA: 0s - loss: 3.7482 - on_off_loss: 2.1572 - dyskinesia_loss: 0.7299 - tremor_loss: 0.8611
Epoch 00001: val_loss improved from inf to 3.35684, saving model to /n/scratch2/ms994/cnnlstm31038.h5
250/250 [==============================] - 142s 567ms/step - loss: 3.7463 - on_off_loss: 2.1566 - dyskinesia_loss: 0.7292 - tremor_loss: 0.8605 - val_loss: 3.3568 - val_on_off_loss: 1.6075 - val_dyskinesia_loss: 0.5952 - val_t

Epoch 18/200
248/250 [============================>.] - ETA: 0s - loss: 1.7931 - on_off_loss: 0.9516 - dyskinesia_loss: 0.3951 - tremor_loss: 0.4464
Epoch 00018: val_loss did not improve from 3.35301
250/250 [==============================] - 12s 47ms/step - loss: 1.7921 - on_off_loss: 0.9513 - dyskinesia_loss: 0.3946 - tremor_loss: 0.4462 - val_loss: 4.4246 - val_on_off_loss: 2.4316 - val_dyskinesia_loss: 0.6449 - val_tremor_loss: 1.3481
Epoch 19/200
248/250 [============================>.] - ETA: 0s - loss: 1.7836 - on_off_loss: 0.9374 - dyskinesia_loss: 0.3970 - tremor_loss: 0.4492
Epoch 00019: val_loss did not improve from 3.35301

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
250/250 [==============================] - 12s 47ms/step - loss: 1.7838 - on_off_loss: 0.9379 - dyskinesia_loss: 0.3972 - tremor_loss: 0.4487 - val_loss: 4.3972 - val_on_off_loss: 2.4162 - val_dyskinesia_loss: 0.6400 - val_tremor_loss: 1.3410
Epoch 20/200
249/250 [==========

Epoch 15/200
249/250 [============================>.] - ETA: 0s - loss: 0.1975 - on_off_loss: 0.0785 - dyskinesia_loss: 0.0612 - tremor_loss: 0.0578
Epoch 00015: val_loss did not improve from 4.00568

Epoch 00015: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
250/250 [==============================] - 12s 47ms/step - loss: 0.1973 - on_off_loss: 0.0784 - dyskinesia_loss: 0.0612 - tremor_loss: 0.0578 - val_loss: 5.3396 - val_on_off_loss: 3.4360 - val_dyskinesia_loss: 1.3042 - val_tremor_loss: 0.5994
Epoch 16/200
248/250 [============================>.] - ETA: 0s - loss: 0.1951 - on_off_loss: 0.0773 - dyskinesia_loss: 0.0586 - tremor_loss: 0.0592
Epoch 00016: val_loss did not improve from 4.00568
250/250 [==============================] - 12s 47ms/step - loss: 0.1955 - on_off_loss: 0.0775 - dyskinesia_loss: 0.0587 - tremor_loss: 0.0592 - val_loss: 5.3267 - val_on_off_loss: 3.4169 - val_dyskinesia_loss: 1.3093 - val_tremor_loss: 0.6005
Epoch 17/200
249/250 [===========

IndexError: list index out of range

In [ ]:
results = []
for subject in all_subjects:
    model = tf.keras.models.load_model(f"/n/scratch2/ms994/cnnlstm3{subject}.h5")
    train_indices, valid_indices, test_indices = tf_mapper.get_splits_per_subject(subject)
    test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(test_indices), batch_size=512, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    results.append(model.evaluate(test_data, steps=1000))

   3/1000 [..............................] - ETA: 3:34:23 - loss: 3.4887 - on_off_loss: 1.2415 - dyskinesia_loss: 1.0925 - tremor_loss: 1.1547 